## 연습문제: 은행 송금


은행은 자신이 계정을 개설하여, 입출금을 할 수 있다.
컨트랙을 프로그램하여 아래 기능을 실행하도록 하세요.
주피터 노트북으로 프로그램하세요.

은행 컨트랙의 함수는 다음과 같이 만드세요.

* 입금액을 정해서 입금
	function deposit(uint amount) public payable
* 전액 인출 함수
	function widthdrawAll() public
* 컨트랙 잔고확인 함수 (this를 이용한 잔고, 상태변수 잔고 (this 잔고가 맞는지 확인하는 용도)
	function getBalance() public view returns(uint, uint)
* 다른 계정으로 계좌이체 함수
	function forwardTo(address payable _receiver) public payable



아래 기능을 실행해 보세요.

- 입금 11111 wei, 222 wei를 하고 잔고 11333 wei 출력

- 자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙 잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력

- 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)

## 1단계 : 컨트랙개발

In [2]:
%%writefile src/Bank.sol

pragma solidity ^0.5.0;

contract Bank {
    address payable owner; //sender
    uint balance; //contract balance
    constructor() public {
        owner = msg.sender;
        balance = address(this).balance; //contract balance
    }
    
    // 입금액을 정해서 입금하는 함수
    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }
    
    // 전액 인출 함수
    function widthdrawAll() public {
        owner.transfer(address(this).balance);
    }
    
    // 컨트랙 잔고확인 함수 (this를 이용한 잔고, 상태변수 잔고 (this 잔고가 맞는지 확인하는 용도)
    function getBalance() public view returns(uint,uint) {
        return (address(this).balance, balance); //now contract address balance
    }

    /*// owner잔고확인 함수
    function getBalanceOfOwner() public view returns (uint) {
        return owner.balance;
    }*/
    
    // 다른 계정으로 계좌이체 함수
    function forwardTo(address payable _receiver) public payable {
        _receiver.transfer(333);
    }
}

Overwriting src/Bank.sol


## 2단계 : 컴파일

In [3]:
!solc --abi --bin src/Bank.sol


======= src/Bank.sol:Bank =======
Binary:
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555047600181905550610218806100676000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad88146100765780633c459375146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf61015a565b005b6100fd600480360360208110156100e757600080fd5b81019080803590602001909291905050506101c4565b005b60008047600154915091509091565b8073ffffffffffffffffffffffffffffffffffffffff166108fc61014d9081150290604051600060405180830381858888f19350505050158015610156573d6000803e3d6000fd5b5050565b6000809054906101000a900473fffffffffffffffffffffffffffffff

In [4]:
!solc --gas src/Bank.sol


======= src/Bank.sol:Bank =======
Gas estimation:
construction:
   41035 + 107200 = 148235
external:
   deposit(uint256):	21103
   forwardTo(address):	infinite
   getBalance():	1027
   widthdrawAll():	infinite


## 3단계 : 배포

In [61]:
%%writefile src/BankDeploy.js

//ganache
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
}
var _abiArray=[{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"constant":false,"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}];
var _bin="608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555047600181905550610218806100676000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad88146100765780633c459375146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf61015a565b005b6100fd600480360360208110156100e757600080fd5b81019080803590602001909291905050506101c4565b005b60008047600154915091509091565b8073ffffffffffffffffffffffffffffffffffffffff166108fc61014d9081150290604051600060405180830381858888f19350505050158015610156573d6000803e3d6000fd5b5050565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166108fc479081150290604051600060405180830381858888f193505050501580156101c1573d6000803e3d6000fd5b50565b8034146101d057600080fd5b806001600082825401925050819055505056fea265627a7a72315820581c73853680d28134d91d1f07c2b389de89dc030d6b72e7b1fb96375734e9d364736f6c63430005100032";
var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0x7954C6dD0459cB8e17361738c49e15593aE189ec", gas: 364124, gasPrice: '1000000000'})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/BankDeploy.js


In [62]:
!node src/BankDeploy.js

0xeBA4a31a432c4Cd54bD0b123e85264CecaCf1f90


## 4단계 : 사용

In [71]:
%%writefile src/BankUse11.js

//ganache
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"constant":false,"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray,"0xeBA4a31a432c4Cd54bD0b123e85264CecaCf1f90");
bank.methods.deposit(11111).send({from:"0x7954C6dD0459cB8e17361738c49e15593aE189ec",gas:800000,value:11111});
bank.methods.deposit(222).send({from:"0x7954C6dD0459cB8e17361738c49e15593aE189ec",gas:800000,value:222});

Overwriting src/BankUse11.js


In [72]:
!node src/BankUse11.js

In [73]:
%%writefile src/BankUse12.js

//ganache
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"constant":false,"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray,"0xeBA4a31a432c4Cd54bD0b123e85264CecaCf1f90");
bank.methods.getBalance().call().then(function(value) {console.log(value);});

Overwriting src/BankUse12.js


In [74]:
!node src/BankUse12.js

Result { '0': '11333', '1': '11333' }


In [75]:
%%writefile src/BankUse21.js

//ganache
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"constant":false,"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray,"0xeBA4a31a432c4Cd54bD0b123e85264CecaCf1f90");
bank.methods.forwardTo("0x12c6B01408c8cFdEE47bC8e5FF64A597C59fCbc0").send({from:"0x7954C6dD0459cB8e17361738c49e15593aE189ec",gas:100000});

Overwriting src/BankUse21.js


In [76]:
!node src/BankUse21.js

In [77]:
%%writefile src/BankUse22.js

//ganache
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"constant":false,"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray,"0xeBA4a31a432c4Cd54bD0b123e85264CecaCf1f90");
bank.methods.getBalance().call().then(function(value) {console.log(value);});

Overwriting src/BankUse22.js


In [78]:
!node src/BankUse22.js

Result { '0': '11000', '1': '11333' }


In [79]:
%%writefile src/BankUse31.js

//ganache
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"constant":false,"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray,"0xeBA4a31a432c4Cd54bD0b123e85264CecaCf1f90");
bank.methods.widthdrawAll().send({from:"0x7954C6dD0459cB8e17361738c49e15593aE189ec",gas:800000});

Overwriting src/BankUse31.js


In [80]:
!node src/BankUse31.js

In [81]:
%%writefile src/BankUse32.js

//ganache
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"constant":false,"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray,"0xeBA4a31a432c4Cd54bD0b123e85264CecaCf1f90");
bank.methods.getBalance().call().then(function(value) {console.log(value);});

Overwriting src/BankUse32.js


In [82]:
!node src/BankUse32.js

Result { '0': '0', '1': '11333' }


### Geth로 오류가 난 이유..?
솔직히 말하면 정확한 이유를 모르겠다. 배포 단계에서 계속 gas도 gasPrice도 바꿔보았음에도 정말 단 한번도 성공한 적이 없다. 금요일 밤부터 이 오류가 있었는데 일요일이 되었는데도 아무것도 해결하지 못하였다..ㅎ 나만 계속 오류가 발생하는 것인지 궁금하여 API수업을 듣는 몇몇 친구들에게 혹시 geth로 성공했냐고 물어보고, 성공했다고 하면 gas와 gasPrice값을 어떻게 바꾸어주었는지 물어봤다. 나도 같은 수로 바꿔보았음에도 불구하고 계속 같은 오류가 났다. 리믹스 코드가 문제인 것인가 싶어 계속 리믹스 코드를 수정해보기도 했다. 심지어 리믹스로도 컴파일은 되나 전액인출이나 문제에서 의도하는 대로 구현되지 않은 코드로 주피터 노트북에 복사하여 실행했다는 학우도, geth로 배포했을 때 오류없이 배포에 성공했다고 했었다. 

교수님 진짜 무엇이 문제인지 모르겠습니다..ㅠㅠ 솔리디티 버전이며 genesis며 모두 수정해보고 그랬는데도 되지를 않네요..ㅠㅠ 